<a href="https://colab.research.google.com/github/JY522IC/PointCloudProject/blob/main/Pointnet_Segment_0612_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The original Pointnet work.
Based on the paper:https://openaccess.thecvf.com/content_cvpr_2017/papers/Qi_PointNet_Deep_Learning_CVPR_2017_paper.pdf
Code:https://github.com/Young98CN/pointconv_pytorch
Github: https://github.com/yanx27/Pointnet_Pointnet2_pytorch/blob/master/models/pointnet_utils.py


# Install the requred libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install the required libraries
!pip install open3d
!pip install provider

# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper functions for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_mesh(pos, face):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    # ax = fig.gca(projection='3d')
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    ax.axes.zaxis.set_ticklabels([])
    ax.plot_trisurf(pos[:, 0], pos[:, 1], pos[:, 2], triangles=data.face.t(), antialiased=False)
    plt.show()


def visualize_points(pos, edge_index=None, index=None):
    fig = plt.figure(figsize=(4, 4))
    if edge_index is not None:
        for (src, dst) in edge_index.t().tolist():
             src = pos[src].tolist()
             dst = pos[dst].tolist()
             plt.plot([src[0], dst[0]], [src[1], dst[1]], linewidth=1, color='black')
    if index is None:
        plt.scatter(pos[:, 0], pos[:, 1], s=50, zorder=1000)
    else:
       mask = torch.zeros(pos.size(0), dtype=torch.bool)
       mask[index] = True
       plt.scatter(pos[~mask, 0], pos[~mask, 1], s=50, color='lightgray', zorder=1000)
       plt.scatter(pos[mask, 0], pos[mask, 1], s=50, zorder=1000)
    plt.axis('off')
    plt.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.4
    Uninstalling widgetsnbextension-3.6.4:
      Successfully uninstalled widgetsnbextension-3.6.4
  Attempting uninstall: werkzeug
    Found existing installa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.5 MB/s eta 0:00:00
2.0.1+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Make the custom dataset

In [ ]:
import torch
import os
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from tqdm import tqdm
import open3d as o3d
import numpy as np
import json
import numpy.linalg

In [ ]:
def inCube(X, corners):
  """
  Check if a point `X` is inside of rectangular prison with `corners` (two points)
  """
  # Where is X > corners?
  greater = X > corners
  # If X is greater than both corners of any axis, it is outside
  inside = ~np.any(np.equal(*greater))

  return inside

In [ ]:
def pc_normalize(pc):
    """
    Point data normilization
    :param pc: the pc to normilized
    :return: the point data after normilization
    """
    # the average value
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    # rescale
    pc = pc / m
    return pc

In [ ]:
# 计算旋转后坐标，以及旋转后点是否在立方体中
#coners是矩形框的8个角的坐标
#我的点的顺序是：后左上，后右上，后右下，后左下，前左上，前右上，前右下，前左下
#points是所有点，判断这些点是否在空间矩形框内
def isin(coners, point):

  in_or_not = 0

  # p = np.array(coners).T
  p = coners
  result = np.array([[1], [1], [1], [1]])

  coefficient = p[:4,]#后面
  ABC1 = np.dot(np.dot(np.linalg.inv(np.dot(coefficient.T, coefficient)), coefficient.T), result)
  coefficient = p[4:,]#前面
  ABC2 = np.dot(np.dot(np.linalg.inv(np.dot(coefficient.T, coefficient)), coefficient.T), result)
  #判断法线是否同向，内积与向量符号一致
  dot_product = np.inner(ABC1.reshape(3), ABC2.reshape(3))
  # dot_product = np.inner(ABC1.reshape(3), ABC2.reshape(3))/numpy.linalg.norm(ABC1.reshape(3))/numpy.linalg.norm(ABC2.reshape(3))
  # angle = np.arccos(dot_product)/np.pi*180
  if (dot_product < 0):
      ABC1 = -ABC1

  coefficient = p[[0,3,4,7],]#左面
  ABC3 = np.dot(np.dot(np.linalg.inv(np.dot(coefficient.T, coefficient)), coefficient.T), result)
  coefficient = p[[1,2,5,6],]#右面
  ABC4 = np.dot(np.dot(np.linalg.inv(np.dot(coefficient.T, coefficient)), coefficient.T), result)
  dot_product = np.inner(ABC3.reshape(3), ABC4.reshape(3))
  if (dot_product < 0):
      ABC3 = -ABC3

  coefficient = p[[0,1,4,5],]#上面
  ABC5 = np.dot(np.dot(np.linalg.inv(np.dot(coefficient.T, coefficient)), coefficient.T), result)
  coefficient = p[[2,3,6,7],]#下面
  ABC6 = np.dot(np.dot(np.linalg.inv(np.dot(coefficient.T, coefficient)), coefficient.T), result)
  dot_product = np.inner(ABC5.reshape(3), ABC6.reshape(3))
  if (dot_product < 0):
      ABC5 = -ABC5
  #p1，p2作为法线的基点
  p1 = p[0,]
  p2 = p[6,]

  p1p = p1 - point
  p2p = p2 - point
  direct1 = np.inner(ABC1.reshape(3), p1p.reshape(3))
  direct2 = np.inner(ABC2.reshape(3), p2p.reshape(3))
  if ((direct1>0 and direct2<0) or (direct1<0 and direct2>0)):
      direct1 = np.inner(ABC3.reshape(3), p1p.reshape(3))
      direct2 = np.inner(ABC4.reshape(3), p2p.reshape(3))
      if ((direct1>0 and direct2<0) or (direct1<0 and direct2>0)):
          direct1 = np.inner(ABC5.reshape(3), p1p.reshape(3))
          direct2 = np.inner(ABC6.reshape(3), p2p.reshape(3))
          if ((direct1>0 and direct2<0) or (direct1<0 and direct2>0)):
              in_or_not = 1
  return in_or_not

def rotate_Y(point, beta):
  point_new = []
  point_new.append(np.cos(beta)*point[0] + np.sin(beta)*point[2])
  point_new.append(point[1])
  point_new.append(-np.sin(beta)*point[0] + np.cos(beta)*point[2])
  return point_new

In [ ]:
class Operating_room_segment(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(Operating_room_segment, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        # List of the raw files
        return []

    @property
    def processed_file_names(self):
        # return list of files should be in processed dir, if found - skip processing
        return ['data.pt']

    # def download(self):
    #     # Download the file specified in self.url and store
    #     # it in self.raw_dir
    #     path = download_url(self.url, self.raw_dir)
    #     extract_zip(path, self.raw_dir)
    #     # The zip file is removed
    #     os.unlink(path)


    def process(self):

      # In the loop we extract the nodes' embeddings, edges connectivity for
      # and label for a graph, process the information and put it in a Data
      # object, then we add the object to a list
      data_list = []
      root_new = '/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Segment0612/import-pointclouds-ply'

      # load the annotations
      annotation_root = os.path.join(root_new, 'annotation.json')
      with open(annotation_root, 'r') as fcc_file:
        annotation = json.load(fcc_file)

      # load the annotations
      frame_pointcloud_map_root = os.path.join(root_new, 'frame_pointcloud_map.json')
      with open(frame_pointcloud_map_root, 'r') as fcc_file:
        frame_pointcloud_map = json.load(fcc_file)

      # Define the direction for the point cloud saved
      pointcloud_data_root = os.path.join(root_new, 'pointcloud')

      # Define the device and dtype to create new point cloud data
      device = o3d.core.Device("CPU:0")
      dtype = o3d.core.float64



      Classes = ['Sitting','Floor', 'Operating table',
      'Monitor','Standing','Chair']
      # Classes = ['Wall','Surgeon sitting on chair', 'Surgeon Standing','Chair',
      # 'Table1','Table2','Table3','Table4']
      Color_class_map = {'Floor': [255, 0, 0], 'Sitting': [192, 192, 192], 'Operating table': [0, 0, 255],
                        'Monitor': [255, 0, 255], 'Standing': [0, 255, 255], 'Chair': [221, 160, 221]}
      Class_title_map = {}
      for i in annotation['objects']:
        Class_title_map[i['key']] = i['classTitle']

      Class_label_map = {
          'Standing': 0,
          'Sitting': 1,
          'Operating table': 2,
          'Monitor': 3,
          'Chair': 4,
          'Floor': 5}

      Label_Class_map = {}
      for i, cat in enumerate(Class_label_map.keys()):
          Label_Class_map[i] = cat

      # Do the point cloud data process
      number_of_files = annotation['framesCount']

      # Define the labelweights
      labelweights = np.zeros(len(Classes))

      for i in range(number_of_files):

        # Define the file path
        file_index = annotation['frames'][i]['index']
        file_name = frame_pointcloud_map[str(file_index)]
        file_dir = os.path.join(root_new,'pointcloud', file_name)

        # Load the pcd file
        pcd = o3d.io.read_point_cloud(file_dir)

        # Control the number of points in the point cloud file
        pcd = pcd.farthest_point_down_sample(15000)

        point_pos = np.asarray(pcd.points)
        point_color = np.asarray(pcd.colors)

        # Define the ground truth colors matrix
        colors_ground_truth_matrix = np.full(np.shape(point_pos), fill_value=192)
        Y_labels = np.zeros(np.shape(point_pos)[0])


        Classes_included = []
        Classes_and_coordinate_information = {}
        itar_value = 0.1
        for k in annotation['frames'][i]['figures']:
          if (int(Class_label_map[Class_title_map[k['objectKey']]]) in Classes_included):
            value = int(Class_label_map[Class_title_map[k['objectKey']]]) + itar_value
            itar_value += 0.1
          else:
            value = int(Class_label_map[Class_title_map[k['objectKey']]])
          Classes_included.append(value)
          Classes_and_coordinate_information[value] = k['geometry']

        # print(Classes_included)
        Classes_included.sort()
        # print(Classes_and_coordinate_information)


        # Labels
        for j in range(np.shape(point_pos)[0]):

            # Get points position
            X = np.array(point_pos[j])

            for k in Classes_included:

              x_min = Classes_and_coordinate_information[k]['position']['x'] - Classes_and_coordinate_information[k]['dimensions']['x']/2
              x_max = Classes_and_coordinate_information[k]['position']['x'] + Classes_and_coordinate_information[k]['dimensions']['x']/2

              y_min = Classes_and_coordinate_information[k]['position']['y'] - Classes_and_coordinate_information[k]['dimensions']['y']/2
              y_max = Classes_and_coordinate_information[k]['position']['y'] + Classes_and_coordinate_information[k]['dimensions']['y']/2

              z_min = Classes_and_coordinate_information[k]['position']['z'] - Classes_and_coordinate_information[k]['dimensions']['z']/2
              z_max = Classes_and_coordinate_information[k]['position']['z'] + Classes_and_coordinate_information[k]['dimensions']['z']/2

              if Classes_and_coordinate_information[k]['rotation']['y'] != 0:

                corners = np.array([
                          [x_min,y_min,z_max],
                          [x_min,y_max,z_max],
                          [x_min,y_max,z_min],
                          [x_min,y_min,z_min],
                          [x_max,y_min,z_max],
                          [x_max,y_max,z_max],
                          [x_max,y_max,z_min],
                          [x_max,y_min,z_min]
                          ])
                for m in range(len(corners)):
                  center_point = [Classes_and_coordinate_information[k]['position']['x'],Classes_and_coordinate_information[k]['position']['y'],Classes_and_coordinate_information[k]['position']['z']]
                  point_xyz = corners[m] - center_point
                  point_xyz = rotate_Y(point_xyz, Classes_and_coordinate_information[k]['rotation']['y'])
                  corners[m] = np.add(point_xyz, center_point)

                if isin(corners, X) == 1:
                  colors_ground_truth_matrix[j] = Color_class_map[Label_Class_map[int(k)]]
                  Y_labels[j] = int(k)

                # https://www.jb51.net/article/276592.htm
                # https://blog.csdn.net/cwm19950318/article/details/114653585

              else:
                corners = [[x_min, y_min, z_min], [x_max, y_max, z_max]]

                if inCube(X, corners):
                  colors_ground_truth_matrix[j] = Color_class_map[Label_Class_map[int(k)]]
                  Y_labels[j] = int(k)


        # Use histogram to calculate the sample weight
        tmp, _ = np.histogram(Y_labels, range(len(Classes)+1))
        # print('tmp')
        #  print(tmp)
        labelweights += tmp

        # Normilize the data
        new_points = pc_normalize(point_pos)

        # Calculate the normal information
        device = o3d.core.Device("CPU:0")
        dtype = o3d.core.float64
        pcd_new = o3d.t.geometry.PointCloud(device)
        pcd_new.point.positions = o3d.core.Tensor(new_points, dtype, device)
        # pcd_new.point.colors = o3d.core.Tensor(colors_ground_truth_matrix, dtype, device)
        pcd_new = pcd_new.to_legacy()
        radius = 0.01  # Searching Radius
        max_nn = 30  # Number of neighboring points to estimate normal vector
        pcd_new.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))
        # o3d.visualization.draw_plotly([pcd_new])

        # Convert the position and label information into the Pytorch format
        Pos = torch.tensor(new_points,dtype=torch.float32)
        Y = torch.tensor(Y_labels,dtype=torch.long)
        Normals = torch.tensor(np.asarray(pcd_new.normals),dtype=torch.float32)
        File_index = torch.tensor(i,dtype=torch.long)
        Colors = torch.tensor(point_color,dtype=torch.float32)
        Colors_ground_truth = torch.tensor(colors_ground_truth_matrix,dtype=torch.float32)

        # Save the data
        graph = Data(pos = Pos, y = Y, normal = Normals, file_index = File_index, colors = Colors, colors_ground_truth = Colors_ground_truth)
        data_list.append(graph)

        print(i)
        #if i >= 2:
        #  break

      # Calculate the sample weight can save in the same index as dataset direction
      labelweights = labelweights.astype(np.float32)
      labelweights = labelweights / np.sum(labelweights)
      labelweights = np.power(np.amax(labelweights) / labelweights, 1 / 3.0)
      # print(labelweights)
      weight_dirction = os.path.join(self.root,'SampleWeightArray.npy')
      np.save(weight_dirction,labelweights)

      # Apply the functions specified in pre_filter and pre_transform
      if self.pre_filter is not None:
          data_list = [data for data in data_list if self.pre_filter(data)]

      if self.pre_transform is not None:
          data_list = [self.pre_transform(data) for data in data_list]

      # Store the processed data
      data, slices = self.collate(data_list)
      torch.save((data, slices), self.processed_paths[0])

In [ ]:
import torch_geometric.transforms as T
# train_index = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/InterestAreaClassfication0521/Datasetmade/InterestArea0523/train0523.csv')
# test_index = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/InterestAreaClassfication0521/Datasetmade/InterestArea0523/test0523.csv')

# Create the train and test dataset based on the direction
# State = 1
trainDataset = Operating_room_segment(root='/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Dataset0612/Train')
# State = 2
testDataset = Operating_room_segment(root='/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Dataset0612/Test')

# Network

In [ ]:
def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self, channel):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32))).view(1, 9).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x

class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1, self.k * self.k).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetEncoder(nn.Module):
    def __init__(self, global_feat=True, feature_transform=False, channel=3):
        super(PointNetEncoder, self).__init__()
        self.stn = STN3d(channel)
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        B, D, N = x.size()
        trans = self.stn(x)
        x = x.transpose(2, 1)
        if D > 3:
            feature = x[:, :, 3:]
            x = x[:, :, :3]
        x = torch.bmm(x, trans)
        if D > 3:
            x = torch.cat([x, feature], dim=2)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2, 1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2, 1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, N)
            return torch.cat([x, pointfeat], 1), trans, trans_feat


def feature_transform_reguliarzer(trans):
    d = trans.size()[1]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2, 1)) - I, dim=(1, 2)))
    return loss

Define the main network structure

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.nn.functional as F

class get_model(nn.Module):
    def __init__(self, num_class):
        super(get_model, self).__init__()
        self.k = num_class
        self.feat = PointNetEncoder(global_feat=False, feature_transform=True, channel=9)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans_feat

class get_loss(torch.nn.Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super(get_loss, self).__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale

    def forward(self, pred, target, trans_feat, weight):
        loss = F.nll_loss(pred, target, weight = weight)
        mat_diff_loss = feature_transform_reguliarzer(trans_feat)
        total_loss = loss + mat_diff_loss * self.mat_diff_loss_scale
        return total_loss

# Define the Main train and test structure

In [ ]:
import os
import sys
import torch
import numpy as np
import time
import datetime
import logging
import provider
import importlib
import shutil
import argparse

from pathlib import Path
from tqdm import tqdm
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

from sklearn.utils.class_weight import compute_sample_weight
from torch_geometric.loader import DataLoader
from torch.utils.data import WeightedRandomSampler

# Define the class and label maps
classes = ['Standing','Sitting','Operating table','Monitor','Chair','Floor']

class2label = {cls: i for i, cls in enumerate(classes)}
seg_classes = class2label
seg_label_to_cat = {}

for i, cat in enumerate(seg_classes.keys()):
    seg_label_to_cat[i] = cat

# Define the required parameters
num_class = len(classes)
# Number of classes in the dataset
optimizer = 'Adam'
# Define the optimizer used
use_cpu = False
# Whether to use cpu
use_normals = False
log_dir = None
# Define the log dir
model = 'pointnet_seg'
# The method name used
learning_rate = 0.001
# Define the learning rate
decay_rate = 1e-4
# Deine the decay rate
epoch = 100
# Define the number of epoch in training
step_size = 10
# Decay step for lr decay [default: every 10 epochs]
lr_decay = 0.7
# Decay rate for lr decay [default: 0.7]
npoint = 15000
# Point number
log_dir = '/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Result/Segment0612/sem_seg/2023-06-13_13-14/'

# Define the log
def log_string(str):
  logger.info(str)
  print(str)

'''HYPER PARAMETER'''
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

'''CREATE DIR'''
timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
# if use_normals:
#   exp_dir = Path('/content/drive/MyDrive/Colab Notebooks/InterestAreaClassfication0521/Results/LogsInterestArea0523/PointnetSixChannel')
# else:
#   exp_dir = Path('/content/drive/MyDrive/Colab Notebooks/InterestAreaClassfication0521/Results/LogsInterestArea0523/Pointnet')
exp_dir = Path('/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Result/Segment0612')
exp_dir.mkdir(exist_ok=True)
exp_dir = exp_dir.joinpath('sem_seg/')
exp_dir.mkdir(exist_ok=True)
if log_dir is None:
    exp_dir = exp_dir.joinpath(timestr)
else:
    exp_dir = exp_dir.joinpath(log_dir)
exp_dir.mkdir(exist_ok=True)
checkpoints_dir = exp_dir.joinpath('checkpoints/')
checkpoints_dir.mkdir(exist_ok=True)
log_dir = exp_dir.joinpath('logs/')
log_dir.mkdir(exist_ok=True)

'''LOG'''
logger = logging.getLogger("Model")
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler('%s/%s.log' % (log_dir, model))
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
log_string('PARAMETER ...')

# Load the data
trainDataLoader = DataLoader(trainDataset, batch_size=10, shuffle=True)
testDataLoader = DataLoader(testDataset, batch_size=10, shuffle=True)

# Load the label weights of training dataset
weights = np.load("/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Dataset0612/Train/SampleWeightArray.npy")
weights = torch.Tensor(weights)

log_string("The number of training data is: %d" % len(trainDataset))
log_string("The number of test data is: %d" % len(testDataset))

# Load the model
classifier = get_model(num_class)
criterion = get_loss()
classifier.apply(inplace_relu)

# if use_cpu == True:
#     classifier = classifier.cuda()
#     criterion = criterion.cuda()

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
        torch.nn.init.xavier_normal_(m.weight.data)
        torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find('Linear') != -1:
        torch.nn.init.xavier_normal_(m.weight.data)
        torch.nn.init.constant_(m.bias.data, 0.0)

try:
    checkpoint = torch.load(str(exp_dir) + '/checkpoints/best_model.pth')
    start_epoch = checkpoint['epoch']
    classifier.load_state_dict(checkpoint['model_state_dict'])
    log_string('Use pretrain model')
except:
    log_string('No existing model, starting training from scratch...')
    start_epoch = 0
    classifier = classifier.apply(weights_init)

if optimizer == 'Adam':
    optimizer = torch.optim.Adam(
        classifier.parameters(),
        lr=learning_rate,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=decay_rate
    )
else:
    optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)

def bn_momentum_adjust(m, momentum):
    if isinstance(m, torch.nn.BatchNorm2d) or isinstance(m, torch.nn.BatchNorm1d):
        m.momentum = momentum

# Training parameters define
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)
global_epoch = 0
global_step = 0
best_instance_acc = 0
best_iou = 0
LEARNING_RATE_CLIP = 1e-5
MOMENTUM_ORIGINAL = 0.1
MOMENTUM_DECCAY = 0.5
MOMENTUM_DECCAY_STEP = step_size

'''TRANING'''
logger.info('Start training...')
for epoch in range(start_epoch, epoch):
    log_string('Epoch %d (%d/%s):' % (global_epoch + 1, epoch + 1, epoch))
    lr = max(learning_rate * (lr_decay ** (epoch // step_size)), LEARNING_RATE_CLIP)
    log_string('Learning rate:%f' % lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    momentum = MOMENTUM_ORIGINAL * (MOMENTUM_DECCAY ** (epoch // MOMENTUM_DECCAY_STEP))
    if momentum < 0.01:
        momentum = 0.01
    print('BN momentum updated to: %f' % momentum)
    classifier = classifier.apply(lambda x: bn_momentum_adjust(x, momentum))


    total_correct = 0
    total_seen = 0
    loss_sum = 0
    classifier = classifier.train()


    # scheduler.step()
    for data in tqdm(trainDataLoader, desc='Processing'):
    #for batch_id, (points, target) in tqdm(enumerate(trainDataLoader, 0), total=len(trainDataLoader), smoothing=0.9):
        optimizer.zero_grad()

        # points = points.data.numpy()
        # print(np.shape(points))
        # points = provider.random_point_dropout(points)
        # points[:, :, 0:3] = provider.random_scale_point_cloud(points[:, :, 0:3])
        # points[:, :, 0:3] = provider.shift_point_cloud(points[:, :, 0:3])
        # if use_normals:
        #   points = data.pos
        #   points = np.asarray(points)
        #   normal = data.normal

        #   points = np.reshape(points,[int(np.shape(points)[0]/5000),5000,3])
        #   normal = np.reshape(normal,[int(np.shape(normal)[0]/5000),5000,3])
        #   new_points = np.concatenate((points,normal),axis=2)
        #   points = torch.Tensor(new_points)
        #   points = points.transpose(2, 1)

        # else:
        #   points = data.pos
        #   points = np.asarray(points)

        #   points = np.reshape(points,[int(np.shape(points)[0]/5000),5000,3])
        #   points = torch.Tensor(points)
        #   points = points.transpose(2, 1)
        points = np.asarray(data.pos)
        color = np.asarray(data.colors)
        normal = np.asarray(data.normal)
        points = np.reshape(points,[int(np.shape(points)[0]/npoint),npoint,3])
        color = np.reshape(color,[int(np.shape(normal)[0]/npoint),npoint,3])
        normal = np.reshape(normal,[int(np.shape(normal)[0]/npoint),npoint,3])
        new_points = np.concatenate((points,color,normal),axis=2)
        points = torch.Tensor(new_points)
        points = points.transpose(2, 1)


        # if not args.use_cpu:
        #     points, target = points.cuda(), target.cuda()

        # print(data.size)

        seg_pred, trans_feat = classifier(points)
        seg_pred = seg_pred.contiguous().view(-1, num_class)

        batch_label = np.asarray(data.y)
        target = data.y
        loss = criterion(seg_pred, target, trans_feat, weights)
        loss.backward()
        optimizer.step()

        pred_choice = seg_pred.cpu().data.max(1)[1].numpy()
        correct = np.sum(pred_choice == batch_label)
        total_correct += correct
        total_seen += (np.shape(points)[0] * npoint)
        loss_sum += loss

    log_string('Training mean loss: %f' % (loss_sum / np.shape(points)[0]))
    log_string('Training accuracy: %f' % (total_correct / float(total_seen)))

    # Save the model every 5 epoch
    if epoch % 5 == 0:
      logger.info('Save model...')
      savepath = str(checkpoints_dir) + '/model.pth'
      log_string('Saving at %s' % savepath)
      state = {
          'epoch': epoch,
          'model_state_dict': classifier.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
      }
      torch.save(state, savepath)
      log_string('Saving model....')

    '''Evaluate on chopped scenes'''
    with torch.no_grad():
      num_batches = len(testDataLoader)
      total_correct = 0
      total_seen = 0
      loss_sum = 0
      labelweights = np.zeros(num_class)
      total_seen_class = [0 for _ in range(num_class)]
      total_correct_class = [0 for _ in range(num_class)]
      total_iou_deno_class = [0 for _ in range(num_class)]
      classifier = classifier.eval()

      log_string('---- EPOCH %03d EVALUATION ----' % (global_epoch + 1))
      for data in tqdm(testDataLoader, desc='Processing'):
        points = np.asarray(data.pos)
        color = np.asarray(data.colors)
        normal = np.asarray(data.normal)
        points = np.reshape(points,[int(np.shape(points)[0]/npoint),npoint,3])
        color = np.reshape(color,[int(np.shape(normal)[0]/npoint),npoint,3])
        normal = np.reshape(normal,[int(np.shape(normal)[0]/npoint),npoint,3])
        new_points = np.concatenate((points,color,normal),axis=2)
        points = torch.Tensor(new_points)
        points = points.transpose(2, 1)

        seg_pred, trans_feat = classifier(points)
        pred_val = seg_pred.contiguous().cpu().data.numpy()
        seg_pred = seg_pred.contiguous().view(-1, num_class)

        batch_label = np.asarray(data.y)
        batch_label = np.reshape(batch_label,[np.shape(points)[0],npoint])
        target = data.y
        loss = criterion(seg_pred, target, trans_feat, weights)
        loss_sum += loss
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum((pred_val == batch_label))
        total_correct += correct
        total_seen += (np.shape(points)[0] * npoint)
        tmp, _ = np.histogram(batch_label, range(num_class + 1))
        labelweights += tmp

        for l in range(num_class):
            total_seen_class[l] += np.sum((batch_label == l))
            total_correct_class[l] += np.sum((pred_val == l) & (batch_label == l))
            total_iou_deno_class[l] += np.sum(((pred_val == l) | (batch_label == l)))
            # true positive / (true positive + false positive + false negative).

      labelweights = labelweights.astype(np.float32) / np.sum(labelweights.astype(np.float32))
      mIoU = np.mean(np.array(total_correct_class) / (np.array(total_iou_deno_class, dtype=np.float) + 1e-6))
      log_string('eval mean loss: %f' % (loss_sum / float(num_batches)))
      log_string('eval point avg class IoU: %f' % (mIoU))
      log_string('eval point accuracy: %f' % (total_correct / float(total_seen)))
      log_string('eval point avg class acc: %f' % (
          np.mean(np.array(total_correct_class) / (np.array(total_seen_class, dtype=np.float) + 1e-6))))

      iou_per_class_str = '------- IoU --------\n'
      for l in range(num_class):
        iou_per_class_str += 'class %s weight: %.3f, IoU: %.3f \n' % (
            seg_label_to_cat[l] + ' ' * (num_class + 1 - len(seg_label_to_cat[l])), labelweights[l - 1],
            total_correct_class[l] / float(total_iou_deno_class[l]))

      log_string(iou_per_class_str)
      log_string('Eval mean loss: %f' % (loss_sum / num_batches))
      log_string('Eval accuracy: %f' % (total_correct / float(total_seen)))

      if mIoU >= best_iou:
          best_iou = mIoU
          logger.info('Save model...')
          savepath = str(checkpoints_dir) + '/best_model.pth'
          log_string('Saving at %s' % savepath)
          state = {
              'epoch': epoch,
              'class_avg_iou': mIoU,
              'model_state_dict': classifier.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
          }
          torch.save(state, savepath)
          log_string('Saving model....')
      log_string('Best mIoU: %f' % best_iou)
    global_epoch += 1

logger.info('End of training...')


<IPython.core.display.Javascript object>

INFO:Model:PARAMETER ...


PARAMETER ...


INFO:Model:The number of training data is: 896
INFO:Model:The number of test data is: 896


The number of training data is: 896
The number of test data is: 896


...

Training mean loss: 1.576100
Training accuracy: 0.971585
---- EPOCH 013 EVALUATION ----


Processing: 100%|██████████| 90/90 [20:12<00:00, 13.48s/it]
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/usr/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/pla

eval mean loss: 1.849388
eval point avg class IoU: 0.462880
eval point accuracy: 0.680063
eval point avg class acc: 0.663513
------- IoU --------
class Standing weight: 0.340, IoU: 0.426 
class Sitting weight: 0.221, IoU: 0.381 
class Operating table weight: 0.016, IoU: 0.042 
class Monitor weight: 0.203, IoU: 0.487 
class Chair   weight: 0.214, IoU: 0.534 
class Floor   weight: 0.006, IoU: 0.906 

Eval mean loss: 1.849388
Eval accuracy: 0.680063
Best mIoU: 0.838025
Epoch 14 (18/17):
Learning rate:0.000700
BN momentum updated to: 0.050000


Processing:  73%|███████▎  | 66/90 [40:38<14:46, 36.95s/it]


KeyboardInterrupt: ignored

# Test and visualization

In [ ]:
import os
import sys
import torch
import numpy as np
import time
import datetime
import logging
import provider
import importlib
import shutil
import argparse
from pathlib import Path
from tqdm import tqdm
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
from sklearn.utils.class_weight import compute_sample_weight
from torch_geometric.loader import DataLoader


# Define the class and label maps
classes = ['Standing','Sitting','Operating table','Monitor','Chair','Floor']

class2label = {cls: i for i, cls in enumerate(classes)}
seg_classes = class2label
seg_label_to_cat = {}
for i, cat in enumerate(seg_classes.keys()):
    seg_label_to_cat[i] = cat



num_class = len(classes)
# Number of classes in the dataset
npoint = 15000
# Number of points in the point cloud

# Load the label informations
root_new = '/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Segment0612/import-pointclouds-ply'
# load the annotations
annotation_root = os.path.join(root_new, 'annotation.json')
with open(annotation_root, 'r') as fcc_file:
  annotation = json.load(fcc_file)

Color_class_map = {'Floor': [255/255, 0.0, 0.0], 'Sitting': [192/255, 192/255, 192/255], 'Operating table': [0.0, 0.0, 255/255],
                        'Monitor': [255/255, 0.0, 255/255], 'Standing': [0.0, 255/255, 255/255], 'Chair': [221/255, 160/255, 221/255]}
# load the annotations
frame_pointcloud_map_root = os.path.join(root_new, 'frame_pointcloud_map.json')
with open(frame_pointcloud_map_root, 'r') as fcc_file:
  frame_pointcloud_map = json.load(fcc_file)

# Load the data
testDataLoader = DataLoader(testDataset, batch_size=3, shuffle=True)
weights = np.load("/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Dataset0612/Test/SampleWeightArray.npy")
weights = torch.Tensor(weights)

# Load the model
experiment_dir = '/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Result/Segment0612/sem_seg/2023-06-13_13-14'
classifier = get_model(num_class)
checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth')
classifier.load_state_dict(checkpoint['model_state_dict'])
classifier = classifier.eval()
visualization_result_path = experiment_dir + '/Viusalization'
try:
 os.mkdir(visualization_result_path)
except:
    pass

device = o3d.core.Device("CPU:0")
dtype = o3d.core.float64

'''Evaluate on chopped scenes'''
with torch.no_grad():
  num_batches = len(testDataLoader)
  total_correct = 0
  total_seen = 0
  loss_sum = 0
  labelweights = np.zeros(num_class)
  total_seen_class = [0 for _ in range(num_class)]
  total_correct_class = [0 for _ in range(num_class)]
  total_iou_deno_class = [0 for _ in range(num_class)]
  classifier = classifier.eval()

  for data in tqdm(testDataLoader, desc='Processing'):
    points = np.asarray(data.pos)
    color = np.asarray(data.colors)
    normal = np.asarray(data.normal)
    points = np.reshape(points,[int(np.shape(points)[0]/npoint),npoint,3])
    color = np.reshape(color,[int(np.shape(normal)[0]/npoint),npoint,3])
    normal = np.reshape(normal,[int(np.shape(normal)[0]/npoint),npoint,3])
    new_points = np.concatenate((points,color,normal),axis=2)
    points = torch.Tensor(new_points)
    points = points.transpose(2, 1)

    seg_pred, trans_feat = classifier(points)
    pred_val = seg_pred.contiguous().cpu().data.numpy()
    seg_pred = seg_pred.contiguous().view(-1, num_class)

    batch_label = np.asarray(data.y)
    batch_label = np.reshape(batch_label,[np.shape(points)[0],npoint])
    target = data.y
    pred_val = np.argmax(pred_val, 2)
    correct = np.sum((pred_val == batch_label))
    total_correct += correct
    total_seen += (np.shape(points)[0] * npoint)
    tmp, _ = np.histogram(batch_label, range(num_class + 1))
    labelweights += tmp

    for l in range(num_class):
      total_seen_class[l] += np.sum((batch_label == l))
      total_correct_class[l] += np.sum((pred_val == l) & (batch_label == l))
      total_iou_deno_class[l] += np.sum(((pred_val == l) | (batch_label == l)))

    for l in range(np.shape(points)[0]):
      # Try to make the test point cloud saved direction
      path_folder = visualization_result_path + '/' + frame_pointcloud_map[str(int(data[l].file_index))]
      try:
       os.mkdir(path_folder)
      except:
        pass

      # Create a new point cloud object
      pcd_new = o3d.t.geometry.PointCloud(device)
      pcd_new.point.positions = o3d.core.Tensor(np.asarray(data[l].pos), dtype, device)
      # Define the colors matrix
      colors_matrix = np.full(np.shape(np.asarray(data[l].pos)),fill_value=192.0)

      for k in range(len(pred_val[l])):
        colors_matrix[k] = Color_class_map[seg_label_to_cat[pred_val[l][k]]]

      pcd_new.point.colors = o3d.core.Tensor(colors_matrix, dtype, device)
      pcd_new = pcd_new.to_legacy()
      path_folder_predict = path_folder + '/Predicted.ply'
      o3d.io.write_point_cloud(path_folder_predict, pcd_new)

      # Create a new point cloud object
      pcd_new_ground_truth = o3d.t.geometry.PointCloud(device)
      pcd_new_ground_truth.point.positions = o3d.core.Tensor(np.asarray(data[l].pos), dtype, device)
      pcd_new_ground_truth.point.colors = o3d.core.Tensor(np.asarray(data[l].colors_ground_truth)/255, dtype, device)
      pcd_new_ground_truth = pcd_new_ground_truth.to_legacy()
      path_folder_predict = path_folder + '/GroundTruth.ply'
      o3d.io.write_point_cloud(path_folder_predict, pcd_new_ground_truth)


  labelweights = labelweights.astype(np.float32) / np.sum(labelweights.astype(np.float32))
  mIoU = np.mean(np.array(total_correct_class) / (np.array(total_iou_deno_class, dtype=np.float) + 1e-6))
  print('eval point avg class IoU: %f' % (mIoU))
  print('eval point accuracy: %f' % (total_correct / float(total_seen)))
  print('eval point avg class acc: %f' % (
      np.mean(np.array(total_correct_class) / (np.array(total_seen_class, dtype=np.float) + 1e-6))))

  iou_per_class_str = '------- IoU --------\n'
  for l in range(num_class):
    iou_per_class_str += 'class %s weight: %.3f, IoU: %.3f \n' % (
        seg_label_to_cat[l] + ' ' * (num_class + 1 - len(seg_label_to_cat[l])), labelweights[l - 1],
        total_correct_class[l] / float(total_iou_deno_class[l]))

  print(iou_per_class_str)
  print('Eval mean loss: %f' % (loss_sum / num_batches))
  print('Eval accuracy: %f' % (total_correct / float(total_seen)))

<IPython.core.display.Javascript object>

Processing: 100%|██████████| 299/299 [21:36<00:00,  4.34s/it]

eval point avg class IoU: 0.805915
eval point accuracy: 0.924837
eval point avg class acc: 0.901278
------- IoU --------
class Standing weight: 0.340, IoU: 0.789 
class Sitting weight: 0.221, IoU: 0.710 
class Operating table weight: 0.016, IoU: 0.868 
class Monitor weight: 0.203, IoU: 0.837 
class Chair   weight: 0.214, IoU: 0.695 
class Floor   weight: 0.006, IoU: 0.936 

Eval mean loss: 0.000000
Eval accuracy: 0.924837



<ipython-input-16-84db52975f39>:144: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mIoU = np.mean(np.array(total_correct_class) / (np.array(total_iou_deno_class, dtype=np.float) + 1e-6))
<ipython-input-16-84db52975f39>:148: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.mean(np.array(total_correct_class) / (np.array(total_seen_class, d